# GLD 25-Delta Cash-Secured Put Backtest (Jupyter Notebook)

This notebook reproduces and slightly hardens the provided backtest script:
- Fetches **10Y daily data** for GLD and SPY from **IBKR** (with a graceful fallback to Yahoo Finance).
- Prices 25-delta short puts using **Black–Scholes** with a **21d realized vol** proxy.
- Runs a cash-secured put strategy targeting **30 DTE** and **-0.25 delta**, with early profit-taking.
- Computes performance metrics (CAGR, Sharpe, Max Drawdown) and makes a few charts.
- Includes an optional **synthetic data mode** to run fully offline.

> **Note:** For live option IV surface, the notebook uses Yahoo Finance option chains.

In [ ]:
# %% [markdown]
# ## Setup & Imports

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from dataclasses import dataclass

# Try IBKR
try:
    from ib_insync import IB, Stock, util
except Exception as e:
    IB = None
    Stock = None
    util = None

from scipy.stats import norm
from scipy.optimize import brentq
import datetime
import math
import time
import random

# seaborn is optional (not required for plots below)
try:
    import seaborn as sns  # noqa: F401
except Exception:
    pass

In [ ]:
# %% [markdown]
# ## Parameters

TAKE_PROFIT = 0.60   # close at 60% of max profit (i.e., when price is 40% of entry)
initial_capital = 100_000.0
target_dte_days = 30
target_put_delta = -0.25
max_concurrent = 10
commission_per_contract = 0
r_annual = 0.015   # risk-free
q_div = 0.0        # GLD dividend yield ~ 0 (expense ignored)

# Reproducibility
rng_seed = 42
np.random.seed(rng_seed); random.seed(rng_seed)

# Data source controls
USE_IBKR = True          # Try to use IBKR first
USE_YFINANCE_FALLBACK = True
USE_SYNTHETIC_IF_ALL_FAIL = False   # set True to force offline synthetic series

# Optional: logo path for figures (set to a valid PNG path or leave as None to skip)
LOGO_PATH = None

# Output directory for any saved images
OUTPUT_DIR = "/mnt/data/pics"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# %% [markdown]
# ## Data Pull (IBKR → Yahoo Finance → Synthetic Fallback)

import os
import pandas as pd
import numpy as np

adj_close_df = pd.DataFrame()

def fetch_ibkr():
    if IB is None:
        return None
    try:
        ib = IB()
        ib.connect('127.0.0.1', 7497, clientId=1, timeout=10)
        ib_tickers = {
            'GLD': Stock('GLD', 'SMART', 'USD'),
            'SPY': Stock('SPY', 'SMART', 'USD'),
        }
        end_date = ''
        df = pd.DataFrame()
        tmp = {}
        for name, contract in ib_tickers.items():
            print(f"Requesting {name} from IBKR...")
            bars = ib.reqHistoricalData(
                contract,
                endDateTime=end_date,      # '' = now
                durationStr='10 Y',
                barSizeSetting='1 day',
                whatToShow='TRADES',
                useRTH=True,
                formatDate=1
            )
            if not bars:
                print(f"No data returned for {name}")
                continue
            df_stock = util.df(bars)
            df_stock.set_index('date', inplace=True)
            tmp[name] = df_stock['close']
            time.sleep(0.2)
        ib.disconnect()
        if tmp:
            out = pd.DataFrame(tmp).sort_index().ffill().dropna()
            return out
    except Exception as e:
        print("IBKR fetch failed:", e)
    return None

def fetch_yfinance():
    try:
        import yfinance as yf
        tickers = ["GLD", "SPY"]
        data = yf.download(tickers, period="10y", interval="1d", auto_adjust=False, progress=False)
        if isinstance(data.columns, pd.MultiIndex):
            close = data["Close"].copy()
        else:
            close = data.rename(columns={"Close":"GLD"})  # unlikely branch
        close = close.dropna(how="all").ffill().dropna()
        return close
    except Exception as e:
        print("yfinance fetch failed:", e)
        return None

def make_synthetic(start='2015-01-01', end=None, mu_gld=0.06, vol_gld=0.16, mu_spy=0.08, vol_spy=0.18, rho=0.1):
    idx = pd.date_range(start=start, end=end or pd.Timestamp.today(), freq="B")
    n = len(idx)
    cov = np.array([[vol_gld**2, rho*vol_gld*vol_spy],
                    [rho*vol_gld*vol_spy, vol_spy**2]])
    L = np.linalg.cholesky(cov/252)
    z = np.random.randn(n,2) @ L.T
    drift = np.array([mu_gld, mu_spy])/252
    r = drift + z
    prices = np.zeros((n,2))
    prices[0,0] = 180.0
    prices[0,1] = 200.0
    for t in range(1,n):
        prices[t] = prices[t-1] * (1 + r[t])
    df = pd.DataFrame(prices, index=idx, columns=["GLD","SPY"])
    return df

adj_close_df = None
if USE_IBKR:
    adj_close_df = fetch_ibkr()

if adj_close_df is None and USE_YFINANCE_FALLBACK:
    adj_close_df = fetch_yfinance()

if adj_close_df is None and USE_SYNTHETIC_IF_ALL_FAIL:
    adj_close_df = make_synthetic()

if adj_close_df is None or adj_close_df.empty:
    raise RuntimeError("Failed to fetch price data from IBKR and yfinance, and synthetic mode is disabled.")

adj_close_df = adj_close_df.sort_index().ffill().dropna()
display(adj_close_df.tail())

In [ ]:
# %% [markdown]
# ## Black–Scholes Helpers

import math
from scipy.stats import norm
from scipy.optimize import brentq

def bs_put_price(S, K, T, r, q, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(K - S, 0.0)
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma * sigma) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    price = K * math.exp(-r * T) * norm.cdf(-d2) - S * math.exp(-q * T) * norm.cdf(-d1)
    return float(price)

def bs_put_delta(S, K, T, r, q, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return -1.0 if K > S else 0.0
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma * sigma) * T) / (sigma * math.sqrt(T))
    return -math.exp(-q * T) * norm.cdf(-d1)

def invert_iv_from_put_price(S, K, T, r, q, price):
    if T <= 0 or S <= 0 or K <= 0 or price <= 0:
        return np.nan
    def f(s):
        return bs_put_price(S, K, T, r, q, s) - price
    try:
        iv = brentq(f, 1e-6, 5.0, maxiter=200)
        return iv
    except Exception:
        return np.nan

def strike_for_target_put_delta(S, T, r, q, sigma, target_delta=-0.25):
    if S <= 0:
        return np.nan
    lo, hi = max(0.01, 0.5 * S), 1.2 * S
    def g(K):
        return bs_put_delta(S, K, T, r, q, sigma) - target_delta
    try:
        K = brentq(g, lo, hi, maxiter=200)
        return K
    except Exception:
        return 0.90 * S

In [ ]:
# %% [markdown]
# ## IV Proxy (21d Realized Vol)

gld = adj_close_df['GLD']
iv_proxy = gld.pct_change().rolling(21).std() * np.sqrt(252)
iv_proxy = iv_proxy.replace([np.inf, -np.inf], np.nan).ffill().bfill()
iv_proxy.name = 'iv_proxy'
display(iv_proxy.tail())

In [ ]:
# %% [markdown]
# ## Backtest Core

from dataclasses import dataclass

@dataclass
class Position:
    strike: float
    expiry: pd.Timestamp
    qty: int          # -1 for short
    entry_price: float

dates = adj_close_df.index
S_series = adj_close_df['GLD']

cash = initial_capital
positions = []
rows = []

bench_qty = math.floor(initial_capital / S_series.iloc[0])
bench_cash = initial_capital - bench_qty * S_series.iloc[0]

for dt in dates:
    S = float(S_series.loc[dt])
    sigma = float(iv_proxy.reindex([dt]).iloc[0])
    sigma = max(1e-6, sigma)
    r = r_annual
    q = q_div

    # Snap expiry to ~30 DTE (use next trading day at or after dt+30)
    target_exp = dt + pd.Timedelta(days=target_dte_days)
    future_trading = dates[dates >= target_exp]
    expiry_trading = future_trading[0] if len(future_trading) else dates[-1]
    T_years = max(1e-9, (expiry_trading - dt).days / 365.0)

    # ---- Manage open positions (MTM & exits) ----
    new_positions = []
    mtm_value = 0.0

    for pos in positions:
        dte = max(0, (pos.expiry - dt).days)
        T_rem = max(1e-9, dte / 365.0)
        P_today = bs_put_price(S, pos.strike, T_rem, r, q, sigma)

        # Profit-take
        if P_today <= (1 - TAKE_PROFIT) * pos.entry_price:
            cash -= (P_today * 100.0) + commission_per_contract  # buy-to-close
            continue

        # Probabilistic early exit if mildly ITM near expiry
        if dte < 5 and S < pos.strike * 0.98 and random.random() < 0.05:
            cash -= (P_today * 100.0) + commission_per_contract
            continue

        # Expiry settlement
        if dte == 0:
            intrinsic = max(pos.strike - S, 0.0)
            cash -= intrinsic * 100.0
            cash -= commission_per_contract
            continue

        # Keep open
        new_positions.append(pos)
        mtm_value += (-1) * P_today * 100.0  # qty is -1

    positions = new_positions

    # ---- Entry: once per day, respect concurrency & cash-secured ----
    # Compute reserved collateral from still-open positions
    reserved_collateral = sum(p.strike * 100.0 for p in positions)

    if len(positions) < max_concurrent:
        K = strike_for_target_put_delta(S, T_years, r, q, sigma, target_put_delta)
        needed = K * 100.0
        available_cash = cash - reserved_collateral
        if available_cash >= needed + commission_per_contract:
            entry_price = bs_put_price(S, K, T_years, r, q, sigma)
            positions.append(Position(K, expiry_trading, -1, entry_price))
            cash += entry_price * 100.0
            cash -= commission_per_contract
            reserved_collateral += needed

    # Recompute MTM after possible entry
    mtm_value = 0.0
    for pos in positions:
        dte = max(0, (pos.expiry - dt).days)
        T_rem = max(1e-9, dte / 365.0)
        P_today = bs_put_price(S, pos.strike, T_rem, r, q, sigma)
        mtm_value += (-1) * P_today * 100.0

    # Risk-free accrual on idle cash
    daily_rf = (1 + r_annual) ** (1/252) - 1
    cash *= (1 + daily_rf)

    equity = cash + mtm_value
    bench_equity = bench_cash + bench_qty * S

    rows.append({
        'date': dt,
        'price_GLD': S,
        'cash': cash,
        'positions': len(positions),
        'mtm': mtm_value,
        'equity': equity,
        'benchmark_equity': bench_equity
    })

results = pd.DataFrame(rows).set_index('date')
results['equity_ret'] = results['equity'].pct_change().fillna(0.0)
display(results.tail())

In [ ]:
# %% [markdown]
# ## Performance Metrics

years = max(1e-9, len(results) / 252.0)
cagr = (results['equity'].iloc[-1] / results['equity'].iloc[0]) ** (1/years) - 1
sharpe = (results['equity_ret'].mean() - ((1 + r_annual)**(1/252) - 1)  ) / (results['equity_ret'].std() + 1e-12) * np.sqrt(252)
dd = results['equity'] / results['equity'].cummax() - 1.0
max_dd = dd.min()

perf = pd.DataFrame({
    'CAGR':[cagr],
    'Sharpe':[sharpe],
    'MaxDrawdown':[max_dd]
})
display(perf)

In [ ]:
# %% [markdown]
# ## Plots

import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import FuncFormatter

def add_figure_logo(fig,
                    logo_path=None,
                    anchor="br",                # 'br','bl','tr','tl'
                    max_width_frac=0.18,        # ≤ 18% of figure width
                    max_height_frac=0.18,       # ≤ 18% of figure height
                    pad_frac=0.02):             # 2% inset from edges
    if not logo_path or not os.path.exists(logo_path):
        return
    img = mpimg.imread(logo_path)
    h_px, w_px = img.shape[:2]
    dpi = fig.dpi
    fig_w_px = fig.get_size_inches()[0] * dpi
    fig_h_px = fig.get_size_inches()[1] * dpi
    zoom_w = (max_width_frac * fig_w_px) / w_px
    zoom_h = (max_height_frac * fig_h_px) / h_px
    zoom = min(zoom_w, zoom_h)
    oi = OffsetImage(img, zoom=zoom)
    if anchor == "br":
        xy = (1 - pad_frac, 0 + pad_frac); align = (1, 0)
    elif anchor == "bl":
        xy = (0 + pad_frac, 0 + pad_frac); align = (0, 0)
    elif anchor == "tr":
        xy = (1 - pad_frac, 1 - pad_frac); align = (1, 1)
    else:  # 'tl'
        xy = (0 + pad_frac, 1 - pad_frac); align = (0, 1)
    ab = AnnotationBbox(oi, xy, xycoords='figure fraction',
                        frameon=False, box_alignment=align, pad=0)
    fig.add_artist(ab)

# 252d rolling correlation
r = adj_close_df.pct_change().dropna()
roll_corr = r['GLD'].rolling(252).corr(r['SPY'])

fig, ax = plt.subplots()
ax.plot(roll_corr.index, roll_corr.values, label='SPY–GLD 252d Corr')
ax.set_title('SPY–GLD 252-Day Rolling Correlation', size=18)
ax.set_ylabel('Correlation'); ax.set_xlabel('Date')
ax.legend(); fig.tight_layout()
add_figure_logo(fig, LOGO_PATH, anchor="br", max_width_frac=0.08, max_height_frac=0.16, pad_frac=0)
plt.show()

In [ ]:
# %%
# Volatility-matched cumulative performance

ret = pd.DataFrame({
    "strategy": results["equity"].pct_change(),
    "gld_bh":   results["benchmark_equity"].pct_change()
}).dropna()

def ann_vol(x, trading_days=252):
    return x.std() * np.sqrt(trading_days)

strategy_vol = ann_vol(ret["strategy"])
gld_vol      = ann_vol(ret["gld_bh"])
scale = 0.0 if gld_vol == 0 else (strategy_vol / gld_vol)
ret["gld_vol_matched"] = ret["gld_bh"] * scale

cum = (1.0 + ret).cumprod()
cum = cum / cum.iloc[0]  # rebase to 1.0 on first common date

fig, ax = plt.subplots()
ax.plot(cum.index, cum["strategy"].values,        label="Strategy")
ax.plot(cum.index, cum["gld_bh"].values,          label="GLD Buy & Hold")
ax.plot(cum.index, cum["gld_vol_matched"].values, label="GLD Vol-Matched")

ax.set_title("Cumulative Return (Volatility-Matched Benchmark)", size=18)
ax.set_ylabel("Growth of $1")
ax.set_xlabel("Date")
ax.legend()
add_figure_logo(fig, LOGO_PATH, anchor="br", max_width_frac=0.08, max_height_frac=0.16, pad_frac=0)
fig.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## Volatility Surface Snapshot (Yahoo Finance)

import numpy as np
import pandas as pd
from scipy.optimize import brentq
from math import log, sqrt, exp
from scipy.stats import norm

def bs_put_price(S, K, T, r, q, sigma):
    if sigma <= 0 or T <= 0:
        return max(K - S, 0.0)
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

def implied_vol_put(S, K, T, r, q, price):
    try:
        return brentq(lambda sigma: bs_put_price(S, K, T, r, q, sigma) - price,
                      1e-6, 5.0, maxiter=500)
    except ValueError:
        return np.nan

try:
    import yfinance as yf
    ticker = yf.Ticker("GLD")
    S = ticker.history(period="1d")['Close'].iloc[-1]
    expiries = ticker.options[:5]
    r = 0.015
    q = 0.0

    grid = []
    for exp in expiries:
        opt_chain = ticker.option_chain(exp)
        puts = opt_chain.puts
        expiry_dt = pd.to_datetime(exp)
        T = (expiry_dt - pd.Timestamp.today()).days / 365.0
        if T <= 0:
            continue
        for _, row in puts.iterrows():
            K = row['strike']
            bid, ask = row['bid'], row['ask']
            if pd.isna(bid) or pd.isna(ask) or (bid <= 0 and ask <= 0):
                continue
            mid = (bid + ask) / 2.0
            if mid <= 0:
                continue
            iv = implied_vol_put(S, K, T, r, q, mid)
            if iv and 0 < iv < 5:
                grid.append((K / S, T * 365.0, iv))

    if grid:
        arr = np.array(grid, dtype=float)
        X, Y, Z = arr[:, 0], arr[:, 1], arr[:, 2]

        # clip 1–99% for readability
        lo, hi = np.nanpercentile(Z, [1, 99])
        mask = (Z >= lo) & (Z <= hi) & np.isfinite(Z) & np.isfinite(X) & np.isfinite(Y)
        X, Y, Z = X[mask], Y[mask], Z[mask]

        fig = plt.figure(figsize=(9, 6))
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_trisurf(X, Y, Z, linewidth=0.2, antialiased=True, cmap="viridis", alpha=0.9)
        ax.scatter(X, Y, Z, s=8, alpha=0.6)
        ax.set_title("GLD Put Implied Volatility Surface", size=18)
        ax.set_xlabel("Moneyness (K/S)")
        ax.set_ylabel("DTE (days)")
        ax.set_zlabel("IV")
        from matplotlib.ticker import FuncFormatter
        ax.zaxis.set_major_formatter(FuncFormatter(lambda v, _: f"{v*100:.0f}%"))
        cbar = fig.colorbar(surf, ax=ax, shrink=0.7, pad=0.08)
        cbar.set_label("IV (decimal)")
        ax.view_init(elev=25, azim=-130)
        def pad(lo, hi, p=0.03):
            span = hi - lo
            return lo - p*span, hi + p*span
        ax.set_xlim(*pad(X.min(), X.max()))
        ax.set_ylim(*pad(Y.min(), Y.max()))
        ax.set_zlim(*pad(Z.min(), Z.max()))
        fig.tight_layout()
        add_figure_logo(fig, LOGO_PATH, anchor="br", max_width_frac=0.08, max_height_frac=0.16, pad_frac=0)
        plt.show()
    else:
        print("No IV surface data available.")
except Exception as e:
    print("IV surface retrieval failed (likely due to no internet):", e)

In [ ]:
# %% [markdown]
# ## Monthly Return Heatmap

from matplotlib.ticker import FuncFormatter

eq = results['equity'].copy()
eq.index = pd.to_datetime(eq.index)

# 1) Daily → Monthly return
daily_ret = eq.pct_change().dropna()
mret = (1 + daily_ret).resample('M').prod().sub(1.0)

# 2) Pivot: rows=year, cols=month
df = mret.to_frame('ret')
df['Year'] = df.index.year
df['Month'] = df.index.month
month_labels = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
pivot = df.pivot(index='Year', columns='Month', values='ret').reindex(columns=range(1,13))

fig, ax = plt.subplots(figsize=(10,6))
im = ax.imshow(pivot.values, aspect='auto', interpolation='nearest', cmap="RdYlGn", vmin=-0.05, vmax=0.05)

ax.set_xticks(np.arange(12))
ax.set_xticklabels(month_labels)
ax.set_yticks(np.arange(len(pivot.index)))
ax.set_yticklabels(pivot.index.astype(int))
ax.set_title('Cash-secured Put Monthly Returns (%)', pad=12, size=18)

# Colorbar (in %)
cbar = fig.colorbar(im, ax=ax, pad=0.02, shrink=0.9, format=FuncFormatter(lambda v,_: f'{v*100:.0f}%'))
cbar.set_label('Return')

# Annotate each cell with %
vals = pivot.values
for i in range(vals.shape[0]):
    for j in range(vals.shape[1]):
        v = vals[i, j]
        if np.isfinite(v):
            ax.text(j, i, f'{v*100:.2f}', ha='center', va='center', fontsize=8, color='black')

# Overlay mask for missing
mask = pivot.isna().values
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i, j]:
            ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1,
                                       fill=True, facecolor="lightgray", edgecolor="w"))

ax.set_xlabel(''); ax.set_ylabel('')
ax.tick_params(length=0)
ax.set_xticks(np.arange(-.5, 12, 1), minor=True)
ax.set_yticks(np.arange(-.5, len(pivot.index), 1), minor=True)
ax.grid(which='minor', color='w', linewidth=1)
ax.tick_params(which='minor', bottom=False, left=False)
add_figure_logo(fig, LOGO_PATH, anchor="br", max_width_frac=0.08, max_height_frac=0.16, pad_frac=0)
fig.tight_layout()
plt.show()

## Appendix: Environment Notes

- To use **IBKR** data, start TWS or IB Gateway locally and ensure API access is enabled on port `7497`.
- If IBKR is unavailable, the notebook will try **Yahoo Finance** (`yfinance`) for daily closes and option chains.
- To force a fully offline run, set `USE_SYNTHETIC_IF_ALL_FAIL = True` (creates a GBM synthetic GLD/SPY series).
- Set `LOGO_PATH` to a PNG to stamp a logo on figures (or leave as `None` to skip).
- Dependencies you may need to install:
  ```bash
  pip install ib_insync yfinance pandas numpy scipy matplotlib
  ```